In [1]:
import os
import glob
import socket
import pymongo
import getpass
import datetime
from pprint import pprint

# Connect to database                                                                                                                                 
uri = 'mongodb://corrections:%s@xenon1t-daq.lngs.infn.it:27017/run'
uri = uri % os.environ.get('MONGO_CORRECTIONS_PASSWORD')
client = pymongo.MongoClient(uri)
db = client['run']
collection = db['neural_network']

In [7]:
# Sort by latest creation time
cursor = collection.find().sort("calculation_time", -1)

In [8]:
# Get and increment version number
major_version_bump = False
minor_version_bump = True

major = 0
minor = 0
if cursor.count() != 0:
    d = cursor[0]
    major = int(d['version'].split('.')[0])
    minor = int(d['version'].split('.')[1])
if major_version_bump:
    major += 1
    minor = 0
if minor_version_bump:
    minor += 1
version = ("%i.%i"%(major, minor))
print ('Bumping to version:', version)

Bumping to version: 1.6


In [9]:
# Define document to insert
doc = {
    'correction': [
        {'field': 'neural_net_file', 'value': 'XENON1T_NN_v8_mc_v030_SR0_n15.npz', 'min': 0, 'max': 6386},
        {'field': 'neural_net_file', 'value': 'XENON1T_NN_v8_mc_v030_SR1_n15.npz', 'min': 6386, 'max': 18836},
        {'field': 'neural_net_file', 'value': 'XENON1T_NN_v8_mc_v030_20180613_postSR1.npz', 'min': 18836, 'max': float("inf")
}
    ],
    'version': version,
    'calculation_time': datetime.datetime.utcnow(),
    'user': getpass.getuser()
}
pprint(doc)

{'calculation_time': datetime.datetime(2018, 7, 19, 16, 17, 39, 735797),
 'correction': [{'field': 'neural_net_file',
                 'max': 6386,
                 'min': 0,
                 'value': 'XENON1T_NN_v8_mc_v030_SR0_n15.npz'},
                {'field': 'neural_net_file',
                 'max': 18836,
                 'min': 6386,
                 'value': 'XENON1T_NN_v8_mc_v030_SR1_n15.npz'},
                {'field': 'neural_net_file',
                 'max': inf,
                 'min': 18836,
                 'value': 'XENON1T_NN_v8_mc_v030_20180613_postSR1.npz'}],
 'user': 'pdeperio',
 'version': '1.6'}


In [10]:
# Check current docs
for document in cursor:
    pprint(document)

{'_id': ObjectId('5b50b9983c1233a82b633dbe'),
 'calculation_time': datetime.datetime(2018, 7, 19, 16, 17, 21, 597000),
 'correction': [{'field': 'neural_net_file',
                 'max': 6386,
                 'min': 0,
                 'value': 'XENON1T_NN_v8_mc_v030_SR0_n15.npz'},
                {'field': 'neural_net_file',
                 'max': 18836,
                 'min': 6386,
                 'value': 'XENON1T_NN_v8_mc_v030_SR1_n15.npz'},
                {'field': 'neural_net_file',
                 'max': inf,
                 'min': 18836,
                 'value': 'XENON1T_NN_v8_mc_v030_20180613_postSR1.npz'}],
 'user': 'pdeperio',
 'version': '1.5'}
{'_id': ObjectId('59cbb5a1620c7204e6c40ef3'),
 'calculation_time': datetime.datetime(2017, 9, 27, 16, 28, 49, 864000),
 'correction': [{'field': 'neural_net_file',
                 'max': 6386,
                 'min': 0,
                 'value': 'NN_XENON1T_v8_mc_v030_sr0_n15.npz'},
                {'field': 'neural_net_fil

In [6]:
# Insert the doc into DB
collection.insert(doc)

/project/lgrandi/anaconda3/envs/pax_head/lib/python3.4/site-packages/ipykernel/__main__.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  from ipykernel import kernelapp as app


ObjectId('5b50b9983c1233a82b633dbe')